# M2E18: Machine Learning Exercises 3 - Sample Solution

---

## Task 1: Finding Patterns in Movies via Clustering Analysis

Recall the movie dataset we used for the **Blockbuster Prediction** project. During the project, you had to clean and transform movie data to build a classification model. In this task, you will
analyze this dataset from an unsupervised learning perspective. 

Your task is to apply the KMeans algorithm to find groups (or clusters) of movies that share similar characteristics. Make sure to:

1. Describe all the data transformation and feature engineering steps performed along the way;
2. Report how the number of clusters impact the results (use the Elbow Method described below); and
3. Show common characteristics of movies grouped into the different clusters.

Feel free to use Kaggle for inspiration. For instance, check this notebook for [an example of clustering analysis](https://www.kaggle.com/kashnitsky/topic-7-unsupervised-learning-pca-and-clustering).

**Elbow Method: Choosing the number of clusters for KMeans**

The elbow method is a heuristic employed for determining KMeans's hyperparameter `k`. The method consists of plotting the explained variation as a function of the number of clusters, and picking the elbow of the curve as the number of clusters to be used. By plotting the elbow, we can find a cutoff point where increasing the number of clusters doesn't give much better results. 

The elbow curve for KMeans can be produced using the code block below:

```python
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

k_values = [k for k in range(1, 11)]
distortions = []

for k in k_values:
    km = KMeans(n_clusters=k)
    km.fit(X)
    distortions.append(km.inertia_)
    
plt.plot(k_values, distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.xticks(k_values)
plt.tight_layout()
plt.show()
```
Note that the same method can be used, for instance, to find the number of principal components to describe a data set in PCA.

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv(os.path.join('Data', 'moviedataset', 'movies_metadata.csv'), index_col='id')

**Sample solution**

1. Describe all the data transformation and feature engineering steps performed along the way:

Filtering irrelevant movies:

In [ ]:
movies.drop(movies[movies['adult'] == 'True'].index, axis='rows', inplace=True)
movies = movies.query("status == 'Released'")

Removing irrelevant attributes:

In [ ]:
movies.drop(
    labels = ['adult', 'overview', 'tagline', 'belongs_to_collection', 
              'homepage', 'poster_path', 
              'status', 'video'], 
    axis='columns', 
    inplace=True
)

Handling missing values:

In [ ]:
for column, missing_value_count in movies.isna().sum().to_dict().items():
    if missing_value_count < 100:
        movies.dropna(subset=[column], axis='rows', inplace=True)

In [ ]:
most_frequent_runtime = movies.runtime.mode()[0]
movies['runtime'] = movies.runtime.fillna(most_frequent_runtime)

Adjusting column types:

In [ ]:
movies['budget'] = movies['budget'].astype(float)
movies['popularity'] = movies['popularity'].astype(float)
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y/%m/%d', errors='coerce')
movies['runtime'] = pd.to_timedelta(movies['runtime'], unit='m')

Rearranging columns:

In [ ]:
movies = movies.reindex(columns = ['imdb_id','title','original_title', 'release_date', 'genres', 'runtime',
                                   'original_language','spoken_languages','production_companies','production_countries', 
                                   'budget', 'revenue', 'vote_average','vote_count','popularity'])

Deleting duplicates:

In [ ]:
movies.drop_duplicates('imdb_id', inplace=True)

Feature engineering:

In [ ]:
dfFeatures = movies.copy(deep=True)

In [ ]:
# Extracting month and year
dfFeatures['year_release'] = movies['release_date'].dt.year
dfFeatures['month_release'] = movies['release_date'].dt.month

# Extracting runtime in minutes
dfFeatures['runtime_in_minutes'] = movies['runtime']/np.timedelta64(1, 'm')

# Extracting categorical features
dfFeatures['spoken_languages'] = dfFeatures.spoken_languages.transform(lambda l: l.split(',')[::2][0].split(":")[-1])
dfFeatures['production_countries'] = dfFeatures.production_countries.transform(lambda l: l.split(',')[::2][0].split(":")[-1])

# Computing numerical features from categorical data
dfFeatures['num_spoken_languages'] = dfFeatures.spoken_languages.apply(len)
dfFeatures['num_production_countries'] = dfFeatures.production_countries.apply(len)
dfFeatures['num_production_companies'] = dfFeatures.production_companies.apply(lambda c: len(c.split(',')))

dfFeatures['main_production_company'] = dfFeatures.production_companies.apply(lambda c: c.split(',')[0].split(":")[-1])
dfFeatures['main_production_countries'] = dfFeatures.production_countries.apply(lambda c: c[0])

# Handling unknown values
dfFeatures.main_production_company.replace([''], 'unknown', inplace=True)
dfFeatures.main_production_countries.replace([''], 'unknown', inplace=True)
dfFeatures.genres.replace([''],'unknown', inplace=True) # Convert missing genre to 'unknow'

Dropping original features:

In [ ]:
dfFeatures.drop(
    labels = ['imdb_id', 'original_title', 'release_date', 'runtime', 'spoken_languages', 
              'production_countries', 'production_companies'], 
    axis='columns', 
    inplace=True
)

Defining X as the dataset for modeling (using only nummeric features):

In [ ]:
X = dfFeatures[dfFeatures.select_dtypes(include=[np.number]).columns].values 

Feature scaling:

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(X) 

2. Report how the number of clusters impact the results (Elbow Method):

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

k_values = [k for k in range(1, 11)]
distortions = []

for k in k_values:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    distortions.append(km.inertia_)
    
plt.plot(k_values, distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.xticks(k_values)
plt.tight_layout()
plt.grid()
plt.show()

From the elbow curve, we can see that `k=4` seems to give good results already. Let's train a KMeans model again using this value:

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=4, random_state=42)
km.fit(X_scaled)

3. Show common characteristics of movies grouped into the different clusters:

Adding cluster number to the original dataset:

In [ ]:
# Get cluster number for each movie
clusters = km.predict(X_scaled)

# Adding to each movie in the original dataset
movies['cluster'] = clusters

Plotting cluster distribution (histogram):

In [ ]:
import seaborn as sns

sns.histplot(movies, x='cluster', discrete=True)
plt.xticks([0, 1, 2, 3]);

Adding movie profit to help gaining insights about the clusters produced by KMeans: 

In [ ]:
movies['profit'] = movies['revenue'] - movies['budget']

Aggregating column values:

In [ ]:
avgdf = movies.groupby(by=['cluster']).mean()
avgdf = avgdf.reset_index() # Flatten the dataframe again

In [ ]:
# Styling the DataFrame: Round numbers to two decimal places
pd.options.display.float_format = '{:.2f}'.format

avgdf.head()

**Findings:**

* Cluster 0: Profitable movies, with high vote average.
* Cluster 1: Movie flops with low popularity.
* Cluster 2: Movies with reasonable profit; most common movie profile.
* Cluster 3: Blockbusters.

## Task 2: Image Classification with Deep Learning

Image classification plays an important role in modern applications such as environmental change, agriculture, urban planning, and surveillance, to name a few examples. The goal is to classify the image by assigning it to a specific label.

A classical dataset for image classification, largely used in academic research, is the CIFAR-10 dataset (Canadian Institute For Advanced Research), which consists of 60000 32x32 color images in 10 classes, with 6000 images per class. The 10 different classes represent airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks, as follows:

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class. 

<div style='align: left; text-align:center;'><br/>
    <img src='img/cifar10.png' alt='CIFAR-10 dataset' style="height:350px;"/>
    <span style='display:block;'>CIFAR-10 dataset. Image Source: <a href="https://www.cs.toronto.edu/~kriz/cifar.html" target="_blank">Alex Krizhevsky's homepage</a></span>
    <br/>
</div>

Use the code block below to build a deep learning classifier that accurately classifies images from the CIFAR-10 dataset. Explain the decisions you made for preprocessing the data and constructing the network.

A good approach to try on this task is to build a [Convolutional Neural Network](https://deepai.org/machine-learning-glossary-and-terms/convolutional-neural-network)(CNNs). Check the video below for a short introduction to CNNs. Alternatively, check this [Towards Data Science post](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53) about CNNs.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('x_VrgWTKkiM', width=600, height=350)

Load CIFAR-10 from TensorFlow and Keras:

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Load the data
cifar = keras.datasets.cifar10 
(X_train, y_train), (X_test, y_test) = cifar.load_data()

In [ ]:
# Check dataset shape
X_train.shape, y_train.shape

In [ ]:
# Define class labels
label = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

In [ ]:
# Plot image no 30 in training

plt.figure()
plt.imshow(X_train[30])
plt.colorbar();

In [ ]:
# Add your code here

**Sample solution**

Rescale images to have values between 0 and 1:

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

#again plot
plt.figure()
plt.imshow(X_train[30])
plt.colorbar();

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import BatchNormalization, Conv2D
from tensorflow.keras.layers import MaxPool2D, Activation, MaxPooling2D

#Build Model
model = Sequential()
model.add(Conv2D(64,activation = 'relu',kernel_size = (3,3 )))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, activation = 'relu', kernel_size = (3,3 )))
model.add(BatchNormalization())
model.add(Flatten(input_shape = (32,32,3)))
model.add(Dense(80, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

Here we used ‘Sparse categorical cross-entropy’. Categorical cross-entropy is a loss function for categorical data and Sparse means it does use all possible classes but some of them

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    X_train, 
    y_train, 
    epochs=10, 
    verbose=1, 
    validation_split=0.3, 
    shuffle=True,
    use_multiprocessing=True
)

Let's plot how the model accuracy (and its loss) evolves over the epochs:

In [ ]:
fig = plt.subplots(figsize=(8, 6))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['accuracy'])

plt.title('Training performance')
plt.ylabel('measurement')
plt.xlabel('epoch')
plt.legend(['Training loss', 'Validation loss', 'Accuracy'], loc = 'upper left')

# Define y-axis ticks
plt.yticks([y/10 for y in range(0, 30, 5)])

# Define x-axis limits
plt.xlim(0, 9)

# Show the grid lines
plt.grid()

plt.show();

With the training phase done, we can check the classification performance on the training data:

In [ ]:
y_train_pred = model.predict_classes(X_train)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_train, y_train_pred))

Now, let's use a confusion matrix to evaluate how the trained model performs on test data. First, we predict classes for the test data:

In [ ]:
y_pred = model.predict_classes(X_test)

And we plot the results on a confusion matrix:

In [ ]:
from sklearn.metrics import confusion_matrix

confmatrix = confusion_matrix(y_test, y_pred)

In [ ]:
import seaborn as sns

# Define figure size
fig, ax = plt.subplots(figsize=(7, 7))

sns.heatmap(confmatrix.T, annot=True, fmt='d', cbar=False, cmap='bone')

plt.xlabel('True label')
plt.ylabel('Predicted label');

We can also check the classification performance using the `classification_report` function from sklearn:

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

**Useful links:**

* [Convolutional Neural Networks in Practice](https://towardsdatascience.com/convolutional-neural-networks-in-practice-406426c6c19a).
* [How to Make an Image Classifier in Python using TensorFlow 2 and Keras](https://www.thepythoncode.com/article/image-classification-keras-python).
* [Convolutional Neural Networks Explained: Using PyTorch to Understand CNNs](https://builtin.com/data-science/convolutional-neural-networks-explained).

## Task 3: COVID-19 Data Analysis

The Coronavirus (COVID-19) pandemic has transformed global life in an unprecedented way. As we struggle to adapt to the new conditions and necessary constraints, governments around the world continue to take evidence-based approaches to this crisis that are grounded in data collection efforts. Most of this data is being made publicly available and can be studied in real-time.

Since April 30th, 2020, data scientist [Martin Henze](https://heads0rtai1s.github.io/) has made available a daily updated dataset on COVID-19 cases and deaths reported in Germany. The dataset consists of two main CSV files: `covid_de.csv` and `demographics_de.csv` (available on your `Data/covid19de` directory and extracted from  [Kaggle](https://www.kaggle.com/headsortails/covid19-tracking-germany)).

`covid_de.csv`: COVID-19 cases and deaths. The original data has been collected by Germany's Robert Koch Institute and can be download through the National Platform for Geographic Data (the latter site also hosts an interactive dashboard).

`demographics_de.csv`: General Demographic Data about Germany on the federal state level. Those have been downloaded from Germany's Federal Office for Statistics (Statistisches Bundesamt) through their Open Data platform GENESIS. The data reflects the (most recent available) estimates on 2018-12-31.

**Use the provided dataset to show**:

1. How the number of daily cases evolved;
2. The fatality distribution along with different age groups;
3. COVID cases and deaths demographics per county (e.g., deaths per case, cases per 1000 residents, deaths per 1000 residents, age group count).
4. The ratio between cases and deaths in `SK Muenster`; and
5. How the data could be used for machine learning purposes. Propose and perform a machine learning task on top of this dataset (e.g., clustering, classification, prediction).

**Sample solution**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid") # Set seaborn plot style

Loading the data:

In [ ]:
covdf = pd.read_csv(os.path.join('Data', 'covid19de', 'covid_de.csv'))
covdf.head()

In [ ]:
covdf['date'] = pd.to_datetime(covdf['date']).dt.date

In [ ]:
covdf.info()

In [ ]:
demographicsdf = pd.read_csv(os.path.join('Data', 'covid19de', 'demographics_de.csv'))
demographicsdf.head()

In [ ]:
demographicsdf.info()

Checking for missing data:

In [ ]:
covdf.isnull().sum() # Show missing values

Handling missing gender values with the help of demographic data:

In [ ]:
# Checking gender demographics
gender = demographicsdf.groupby(by='gender').sum()

# Checking gender ration among German population
gender['ratio'] = gender['population']/gender['population'].sum()

# Show gender demographics
gender

In [ ]:
# Get all missing gender values
na_gender = covdf["gender"].isna()

# Impute missing data using gender ratio
female_index = covdf["gender"].loc[na_gender].sample(frac=0.51).index
covdf.loc[female_index, "gender"] = "F" # Impute "female" gender

# Impute the reminder missing gender values as "male" 
covdf["gender"].fillna("M", inplace=True)

Handling missing age-group values:

In [ ]:
# Population size categorized into age groups
demographicsdf.groupby(by="age_group").sum()

In [ ]:
# Finding most frequent age group
most_freq_age_group = demographicsdf.groupby(by="age_group").sum().idxmax()
most_freq_age_group

In [ ]:
# Impute missing age groups with the most frequent one
covdf["age_group"].fillna("35-59", inplace=True)

In [ ]:
# Checking for missing values again
covdf.isnull().sum()

In [ ]:
demographicsdf.isnull().sum() # Show missing values

1. How the number of daily cases evolved:

In [ ]:
daily_cases_df = covdf.groupby("date").sum()
daily_cases_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

plt.title("Daily cases")
sns.lineplot(x='date', y='cases', data=daily_cases_df)
plt.ylabel("Daily cases")

# Show dates on x-axis
#ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1)) 

plt.xlabel('')
plt.xticks(ha='left', rotation=-20)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

plt.title("Daily deaths")
sns.lineplot(data=daily_cases_df["deaths"], color="orange")

plt.xlabel('')
plt.ylabel("Daily deaths")
plt.xticks(ha='left', rotation=-20)

plt.show()

2. The fatality distribution along with different age groups:

In [ ]:
age_covdf = covdf.groupby(by=['age_group', 'gender']).sum().sort_values(by='deaths', ascending=False)
age_covdf = age_covdf.reset_index() # Flatten DataFrame
age_covdf

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

plt.title("Deaths by Age Group")

# Create a bar chart distinguishing counts by gender
psns = sns.barplot(x='age_group', y='deaths', data=age_covdf, hue='gender', palette='gray')

# Add percentage on top of the plot bars
for p in psns.patches:
    psns.annotate(format(100 * p.get_height() / age_covdf["deaths"].sum(), ".1f") + "%", 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = "center", va = "center", 
                   xytext = (0, 9), 
                   textcoords = "offset points")

plt.xlabel('Age Group')
plt.ylabel("Deaths")

plt.show()

3.  COVID cases and deaths demographics per county/state (e.g., deaths per case, cases per 1000 residents, deaths per 1000 residents, age group count):

In [ ]:
# Define function for plotting bar charts
def plot_demographics(x, y, categories, dataframe, title, xlabel, ylabel, rotate_x_ticks=True):
    fig, ax = plt.subplots(figsize=(12, 6))

    plt.title(title)

    # Create a bar chart
    sns.barplot(
        x=x, # x-axis attribute
        y=y, # y-axis attribute
        data=dataframe, # DataFrame
        hue=categories, # Color bars based on data attribute
        palette='tab20', # Color scheme/map 
        dodge=False # Enlarge bars
    )

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    if rotate_x_ticks:
        plt.xticks(ha='left', rotation=90)
    
    plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
    

    plt.show()

In [ ]:
# Group by state
state_agg_df = covdf.groupby(by='state').sum().reset_index()

# Compute deaths per case ratio
state_agg_df['deaths_per_case'] = 100*state_agg_df['deaths']/state_agg_df['cases']

In [ ]:
# Plot bar chart
plot_demographics(
    x='state',
    y='deaths_per_case',
    dataframe=state_agg_df,
    categories='state',
    xlabel='',
    ylabel='In percentage %',
    title='Deaths per case ratio'
)

In [ ]:
# Compute population per state
pop_state_df = demographicsdf[['state', 'population']].groupby(by='state').sum().reset_index()

In [ ]:
# Merge demographics with covid data
state_cov_pop_df = state_agg_df.merge(
    pop_state_df, 
    how='left', 
    left_on='state', 
    right_on='state'
)

state_cov_pop_df

In [ ]:
# Compute cases per 1000 residents
state_cov_pop_df['cases_per_1000'] = state_cov_pop_df['cases'] * (1000/state_cov_pop_df['population'])

In [ ]:
plot_demographics(
    x='state',
    y='cases_per_1000',
    dataframe=state_cov_pop_df,
    categories='state',
    xlabel='',
    ylabel='no. cases per 1000',
    title='Cases per 1000 residents'
)

In [ ]:
# Compute deaths per 1000 residents
state_cov_pop_df['deaths_per_1000'] = state_cov_pop_df['deaths'] * (1000/state_cov_pop_df['population'])

In [ ]:
plot_demographics(
    x='state',
    y='deaths_per_1000',
    dataframe=state_cov_pop_df,
    categories='state',
    xlabel='',
    ylabel='no. deaths per 1000',
    title='Deaths per 1000 residents'
)

4. The ratio between cases and deaths in SK Muenster:

In [ ]:
# Get SK Muenster data
muensterdf = covdf[covdf.county.str.startswith('SK Muenster')]
muensterdf

In [ ]:
# Create a dataframe with overall COVID numbers in Münster
ms_covid_df = pd.melt(
    muensterdf[['date','cases', 'deaths', 'recovered']], 
    'date', 
    var_name='Measurement', 
    value_name='value'
)

In [ ]:
# Plot line chart with cases, deaths, and recovered numbers
fig, ax = plt.subplots(figsize=(12, 5))

plt.title('COVID-19 numbers in Münster')

sns.lineplot(
    x='date', 
    y='value',
    hue='Measurement',
    data=ms_covid_df,
    palette='magma',
    ci=None
)

plt.ylabel('count')
plt.xticks(ha='left', rotation=-20)

plt.show()

In [ ]:
# Group by county
agg_muensterdf = muensterdf.groupby(by='county').sum()

# Compute deaths per case ratio
agg_muensterdf['deaths_per_case'] = 100*agg_muensterdf['deaths']/agg_muensterdf['cases']

print('Deaths per case ratio in SK Münster: ', agg_muensterdf['deaths_per_case'].values)

In [ ]:
# Plot cases by gender
sns.set_style("whitegrid") # Set seaborn plot style

fig, ax = plt.subplots(figsize=(12, 5))

plt.title("Daily cases")

sns.lineplot(
    x='date', 
    y='cases', 
    data=muensterdf.groupby(by=['date', 'gender']).sum().reset_index(), 
    hue='gender',
    palette='magma'
)

plt.ylabel("Daily cases")
plt.xticks(ha='left', rotation=-20)

plt.show()

5. How the dataset could be used for machine learning purposes. Propose and perform a machine learning task on top of this dataset (e.g., clustering, classification, prediction):

**Defining the problem**

Here, we define a clustering problem where we want to find clusters of counties sharing similar characteristics despite their geographic location. Our hypothesis is that counties in different states might share similar patterns, which can disclose interesting insights about the development of COVID-19 cases.

To perform clustering analysis, we first need to engineer features that portray relevant information about COVID-19 cases in different counties. Let's start by...

In [ ]:
covdf.head()

In [ ]:
covdf.groupby(by=['state', 'county', 'age_group']).sum().reset_index().pivot(index='county', columns='age_group', values='cases')

In [ ]:
# Create initial features DataFrame with sum of cases, deaths, and recoveries
featuresdf = covdf.groupby(by=['state', 'county']).sum()

for col in ['cases', 'deaths', 'recovered']:
    # Get overall numbers (i.e., cases, deaths, and recovered) per age group 
    df = covdf.groupby(by=['state', 'county', 'age_group']).sum().reset_index().pivot(
                    index='county', 
                    columns='age_group', 
                    values=col)
    
    # Give explicit names to new features
    df.columns = [f'{col}-{c}-age' for c in df.columns]

    # Merge/join new features with the initial ones
    featuresdf = featuresdf.merge(
        df, 
        how='left', 
        left_on='county', 
        right_on='county')  

In [ ]:
featuresdf.head()

Defining the X variable for our modeling task:

In [ ]:
X = featuresdf.values

Feature scaling:

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(X)

Selecting the number of clusters of KMeans:

In [ ]:
# import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

k_values = [k for k in range(1, 11)]
distortions = []

for k in k_values:
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(X_scaled)
    distortions.append(km.inertia_)
    
plt.plot(k_values, distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.xticks(k_values)
plt.tight_layout()
plt.grid()
plt.show()

`k=4` seems to give good results already:

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=4, random_state=42)
km.fit(X_scaled);

Adding cluster number to counties:

In [ ]:
featuresdf['cluster'] = km.predict(X_scaled)

Plotting cluster distribution (histogram):

In [ ]:
import seaborn as sns

sns.set_style('white')

fig, ax = plt.subplots(figsize=(5, 5))

hpolt = sns.histplot(featuresdf, x='cluster', discrete=True)

# Add percentage on top of the plot bars
for p in hpolt.patches:
    hpolt.annotate(p.get_height(), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = "center", va = "center", 
                   xytext = (0, 9), 
                   textcoords = "offset points")
    
plt.xticks([c for c in range(0, km.n_clusters)])
sns.despine()
plt.show()

Aggregating column values:

In [ ]:
# Styling the DataFrame: Round numbers to two decimal places
pd.options.display.float_format = '{:.2f}'.format

# Aggregating clusters
avgdf = featuresdf.groupby(by=['cluster']).mean()

avgdf.T

**Clustering results (reported numbers in average)**

**Cluster 0**: ~7400 cases, ~160 deaths

In [ ]:
[c for c in featuresdf.query('cluster == 0').index.sort_values()]

**Cluster 1**: ~15555 cases, ~351 deaths

In [ ]:
[c for c in featuresdf.query('cluster == 1').index.sort_values()]

**Cluster 2 (lower number of cases and deaths)**: ~2682 cases, ~65 deaths

In [ ]:
[c for c in featuresdf.query('cluster == 2').index.sort_values()]

**Cluster 3 (higher number of cases and deaths) - Only two counties. Responsible for the majority of cases**: ~45775 cases, ~795 deaths

In [ ]:
[c for c in featuresdf.query('cluster == 3').index.sort_values()]

**Useful links**:

* https://www.kaggle.com/marcoitt/covid19-in-germany-overview
* https://www.kaggle.com/surajkumar88/covid19-case-study-analysis-viz-predictions

---

# End M2E18:  Machine Learning Exercises 3 - Sample Solution